In [250]:
import pandas as pd

df = pd.read_csv('Project.csv')

In [251]:
df.sample(3)

,transaction_id,transaction_date,transaction_time,store_id,store_location,product_id,transaction_qty,unit_price,Total_Bill,product_category,product_type,product_detail,Size,Month Name,Day Name,Hour,Month,Day of Week
104476,123131,08-06-2023,19:27:54,5,Lower Manhattan,48,2,2.50,5.00,Tea,Brewed Black tea,English Breakfast,Regular,June,Thursday,19,6,3
140360,112910,30-05-2023,16:49:21,5,Lower Manhattan,30,2,3.00,6.00,Coffee,Gourmet brewed coffee,Columbian Medium Roast,Large,May,Tuesday,16,5,1
34932,100204,19-05-2023,07:21:11,8,Hell's Kitchen,71,1,3.75,3.75,Bakery,Pastry,Chocolate Croissant,Not Defined,May,Friday,7,5,4


In [252]:
combine_features = df['product_category']+' '+df['product_type']+' '+df['product_detail'] 

In [253]:
df['detail'] = combine_features

In [254]:
df = df[['product_detail', 'detail']]
df['index'] = range(len(df))
df.set_index('index', inplace=True)

In [255]:
seen = set()
df['check'] = df['detail'].apply(lambda x: not (x in seen or seen.add(x)))

In [256]:
df = df.reset_index()

In [257]:
df = df[df['check']==True]

In [258]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=10000, stop_words='english')
feature_vectors = cv.fit_transform(df['detail'].values.astype('U')).toarray()

In [259]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(feature_vectors)

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend(item):
    item_lower = item.lower()

    item_index = df[df['detail'].str.lower().str.contains(item_lower, regex=False)].index
    if len(item_index) == 0:
        print("Item not found in DataFrame")
        return
    index = item_index[0]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['detail'])
    
    similarity = cosine_similarity(tfidf_matrix[index], tfidf_matrix)

    distance = sorted(enumerate(similarity[0]), reverse=True, key=lambda x: x[1])

    print("Recommendations for", item, ":")
    for i in distance[:10]:
        print(f"- Similarity: {i[1]:.4f} | Detail: {df.iloc[i[0]]['product_detail']}")

recommend('bakery')


IndexError: row index (15876) out of range